In [34]:
import pandas as pd
import requests
from bs4 import BeautifulSoup


In [5]:
soup = BeautifulSoup(requests.get("https://www23.statcan.gc.ca/imdb/p3VD.pl?Function=getVD&TVD=53971").content)

In [14]:
state_name_df = pd.DataFrame([[td.text for td in tr.find_all("td")] for tr in soup.find("table").find_all("tr")]).dropna().reset_index(drop=True)
state_name_df.columns = ["name", "abbr_small", "abbr_large"]
abbr_large_to_name_dict = state_name_df.set_index("abbr_large")["name"].to_dict()

In [39]:
def make_code_five_positions(number):
    code = str(number)
    while len(code) < 5:
        code = "0" + code
        
    return code

url_1 = "https://statecancerprofiles.cancer.gov/incidencerates/index.php?stateFIPS=00&areatype=county&cancer=001&stage=999&race=00&sex=0&age=001&year=0&type=incd&sortVariableName=rate&sortOrder=desc&output=1"
file_path_1 = "data_files/incd.csv"

with open(file_path_1, "r") as file:
    lines = file.readlines()

df = pd.read_csv(file_path_1, sep=",")

columns_dict = {'Age-Adjusted Incidence Rate([rate note]) - cases per 100,000':"incidence_rate", "State":"County"}
df.columns = [x.strip() for x in df.columns]
df = df.rename(columns=columns_dict)
df["incidence_rate"] = pd.to_numeric(df["incidence_rate"], errors="coerce")
df = df.dropna(thresh=2)
df = df.sort_values("incidence_rate")
df["State"] = df["County"].apply(lambda x: x.split(",")[-1].strip())
df["State"] = df["State"].apply(lambda x: x.split("(")[0].strip())
df["County"] = df["County"].apply(lambda x: x.split(",")[0].strip())
df["FIPS"] = df["FIPS"].apply(lambda x: str(int(x)))
df["FIPS"] = df["FIPS"].apply(lambda x: make_code_five_positions(x))
#df[df["State"].str.contains("New Jersey")]
#df[df["State"].str.contains("New York")]
#df[df["State"].str.contains("Texas")].iloc[:20]
#df.iloc[:20]

df.set_index(df.columns[0]).to_csv("data_files/county_level_5_year_average_cleaned.csv")

In [45]:
zip_to_fips_df = pd.read_csv("data_files/ZIP-COUNTY-FIPS_2010-03.csv")
zip_to_fips_df["FIPS"] = zip_to_fips_df["STCOUNTYFP"].apply(lambda x: make_code_five_positions(x))
zip_to_fips_dict = zip_to_fips_df.set_index("ZIP")["FIPS"].to_dict()

In [61]:
faciliaty_df = pd.read_excel("data_files/2020_data_summary_spreadsheets/ghgp_data_2020.xlsx", skiprows=2, header=1)
faciliaty_df["State"] = faciliaty_df["State"].apply(lambda x: abbr_large_to_name_dict.get(x, x))

faciliaty_df["FIPS"] = faciliaty_df["Zip Code"].apply(lambda x: zip_to_fips_dict.get(x))  


In [62]:
for index, row in faciliaty_df.iterrows():

       result = row[['Total reported direct emissions', 'CO2 emissions (non-biogenic) ',
              'Methane (CH4) emissions ', 'Nitrous Oxide (N2O) emissions ',
              'HFC emissions', 'PFC emissions', 'SF6 emissions ', 'NF3 emissions',
              'Other Fully Fluorinated GHG emissions', 'HFE emissions',
              'Very Short-lived Compounds emissions', 'Other GHGs (metric tons CO2e)',
              'Biogenic CO2 emissions (metric tons)', 'Stationary Combustion',
              'Electricity Generation', 'Adipic Acid Production',
              'Aluminum Production', 'Ammonia Manufacturing', 'Cement Production',
              'Electronics Manufacture', 'Ferroalloy Production',
              'Fluorinated GHG Production', 'Glass Production',
              'HCFC–22 Production from HFC–23 Destruction', 'Hydrogen Production',
              'Iron and Steel Production', 'Lead Production', 'Lime Production',
              'Magnesium Production', 'Miscellaneous Use of Carbonates',
              'Nitric Acid Production',
              'Petroleum and Natural Gas Systems – Offshore Production',
              'Petroleum and Natural Gas Systems – Processing',
              'Petroleum and Natural Gas Systems – Transmission/Compression',
              'Petroleum and Natural Gas Systems – Underground Storage',
              'Petroleum and Natural Gas Systems – LNG Storage',
              'Petroleum and Natural Gas Systems – LNG Import/Export',
              'Petrochemical Production', 'Petroleum Refining',
              'Phosphoric Acid Production', 'Pulp and Paper Manufacturing',
              'Silicon Carbide Production', 'Soda Ash Manufacturing',
              'Titanium Dioxide Production', 'Underground Coal Mines',
              'Zinc Production', 'Municipal Landfills',
              'Industrial Wastewater Treatment',
              'Manufacture of Electric Transmission and Distribution Equipment',
              'Industrial Waste Landfills']].dropna()
       faciliaty_df.loc[index, "emission_types"] = "<br>".join(result.index)

In [63]:
faciliaty_df = faciliaty_df[['Facility Name', 'City', 'State', 'Zip Code', 'Address', 'County', 'Latitude', 'Longitude', 'Primary NAICS Code', 'Industry Type (subparts)', 'Industry Type (sectors)', 'Total reported direct emissions', 'emission_types']]
faciliaty_df.set_index(faciliaty_df.columns[0]).to_csv("data_files/facilities_1.csv")

In [33]:
len(faciliaty_df.State.unique())

faciliaty_df.State.value_counts()
faciliaty_df.groupby(["Industry Type (sectors)", "State"]).count().iloc[:40]
faciliaty_df.groupby(["Industry Type (sectors)"]).count().sort_values("Facility Name").iloc[-20:]

,Facility Name,City,State,Zip Code,Address,County,Latitude,Longitude,Primary NAICS Code,Industry Type (subparts),Total reported direct emissions
Industry Type (sectors),,,,,,,,,,,
"Injection of CO2,Petroleum and Natural Gas Systems,Suppliers of CO2",10,10,10,10,5,7,10,10,10,10,10
"Other,Suppliers of CO2,Waste",11,11,11,11,11,11,11,11,11,11,11
"Chemicals,Waste",11,11,11,11,11,11,11,11,11,11,11
"Chemicals,Industrial Gas Suppliers",15,15,15,15,15,15,15,15,15,15,15
"Metals,Waste",17,17,17,17,16,17,17,17,17,17,17
"Chemicals,Suppliers of CO2",29,29,29,29,29,29,29,29,29,29,29
"Other,Suppliers of CO2",35,35,35,35,35,35,35,35,35,35,35
"Chemicals,Petroleum Product Suppliers,Refineries",41,41,41,41,41,41,41,41,41,41,41
"Petroleum Product Suppliers,Refineries",69,69,69,69,66,68,69,69,69,69,69


In [104]:
state_choices = ["New York", "New Jersey", "DC", "Virginia", "Maryland", "Delaware", "Connecticut", "Rhode Island", "Massachusetts", "Pennsylvania", "Georgia", "South Carolina", "North Carolina", "West Viriginia", "Kentucky"]

df[df["State"].str.contains("|".join(state_choices))].iloc[:30]


,County,FIPS,Met Healthy People Objective of ***?,incidence_rate,Lower 95% Confidence Interval,Upper 95% Confidence Interval,CI*Rank([rank note]),Lower CI (CI*Rank),Upper CI (CI*Rank),Average Annual Count,Recent Trend,Recent 5-Year Trend ([trend note]) in Incidence Rates,Lower 95% Confidence Interval.1,Upper 95% Confidence Interval.1,State
2933,"Bristol City, Virginia(6)",51520.0,***,214.7,187.9,244.5,N/A,129,133,50,falling,-10.1,-15.0,-5.0,Virginia(6)
2930,"Lee County, Virginia(6)",51105.0,***,231.0,207.9,256.2,N/A,128,133,79,falling,-4.6,-6.9,-2.2,Virginia(6)
2928,"Scott County, Virginia(6)",51169.0,***,232.4,208.8,258.3,N/A,128,133,79,falling,-4.1,-7.0,-1.1,Virginia(6)
2927,"Russell County, Virginia(6)",51167.0,***,238.9,216.8,262.9,N/A,128,133,95,falling,-8.5,-11.6,-5.2,Virginia(6)
2921,"Buchanan County, Virginia(6)",51027.0,***,259.7,234.5,287.3,N/A,125,133,85,falling,-2.7,-4.3,-1.1,Virginia(6)
2900,"Smyth County, Virginia(6)",51173.0,***,285.3,262.7,309.5,N/A,122,130,129,falling,-21.1,-34.0,-5.7,Virginia(6)
2885,"Tazewell County, Virginia(6)",51185.0,***,300.4,280.2,321.8,N/A,119,129,182,falling,-4.5,-6.5,-2.5,Virginia(6)
2881,"Manassas Park City, Virginia(6)",51685.0,***,304.7,260.2,354.2,N/A,104,131,38,stable,-15.6,-39.1,17.0,Virginia(6)
2848,"Telfair County, Georgia(7)",13271.0,***,320.6,286.7,357.8,N/A,148,159,67,stable,-0.8,-1.7,0.1,Georgia(7)
2844,"Wheeler County, Georgia(7)",13309.0,***,323.3,272.8,381.1,N/A,129,159,30,stable,1.4,-1.1,3.9,Georgia(7)


,,,,,,,,,,,,,Sorted by Rate
State,FIPS,Met Healthy People Objective of ***?,"Age-Adjusted Incidence Rate([rate note]) - cases per 100,000",Lower 95% Confidence Interval,Upper 95% Confidence Interval,CI*Rank([rank note]),Lower CI (CI*Rank),Upper CI (CI*Rank),Average Annual Count,Recent Trend,Recent 5-Year Trend ([trend note]) in Incidence Rates,Lower 95% Confidence Interval,Upper 95% Confidence Interval
US (SEER+NPCR)(1),00000,***,448.6,448.3,448.9,N/A,N/A,N/A,1703249,falling,-0.9,-1.1,-0.7
"Union County, Florida(6)",12125,***,1136.4,1066.6,1209.9,N/A,1,1,214,stable,0.0,-1.0,1.1
"Richmond County, Virginia(6)",51159,***,784.2,714.8,859.3,N/A,1,3,99,rising,35.6,20.0,53.2
"Lexington City, Virginia(6)",51678,***,727.5,633.8,832.3,N/A,1,9,52,rising,28.1,6.9,53.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Woodson County, Kansas(6)",20207,***,*,*,*,*,*,*,3 or fewer,*,*,*,*
"Wright County, Minnesota(6)",27171,***,*,*,*,*,*,*,3 or fewer,*,*,*,*
"Wyandotte County, Kansas(6)",20209,***,*,*,*,*,*,*,3 or fewer,*,*,*,*
"Yakutat City and Borough, Alaska(6)",02282,***,*,*,*,*,*,*,3 or fewer,*,*,*,*


Index(['Sorted by Rate'], dtype='object')

Sorted by Rate    Upper 95% Confidence Interval
Name: (State,  FIPS, Met Healthy People Objective of ***?, Age-Adjusted Incidence Rate([rate note]) - cases per 100,000, Lower 95% Confidence Interval, Upper 95% Confidence Interval, CI*Rank([rank note]), Lower CI (CI*Rank), Upper CI (CI*Rank), Average Annual Count, Recent Trend, Recent 5-Year Trend ([trend note]) in Incidence Rates, Lower 95% Confidence Interval), dtype: object